In [1]:
import os

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import Simulator, StartMeshcat

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
def create_scene():
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
"""

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


create_scene()

RuntimeError: error: URI 'package://jengabot/models/table_top.sdf' refers to unknown package 'jengabot'